# Dask Arrays with Xarray

The scientific Python package known as Dask provides Dask Arrays: parallel,larger-than-memory, n-dimensional arrays that make use of blocked algorithms. They are analogous to Numpy arrays, but are distributed. These terms are defined below:

* Parallel code uses many or all of the cores on the computer running the code.

* Larger-than-memory refers to algorithms that break up data arrays into small pieces, operate on these pieces in an optimized fashion, and stream data from a storage device. This allows a user or programmer to work with datasets of a size larger than the available memory.

* A blocked algorithm speeds up large computations by converting them into a series of smaller computations.

In this tutorial, we cover the use of Xarray to wrap Dask arrays. By using Dask arrays instead of Numpy arrays in Xarray data objects, it becomes possible to execute analysis code in parallel with much less code and effort.

# Learning Objectives

* Learn the distinction between eager and lazy execution, and performing both types of execution with Xarray

* Understand key features of Dask Arrays

* Learn to perform operations with Dask Arrays in similar ways to performing operations with NumPy arrays

*  the use of Xarray DataArrays and Datasets as “Dask collections”, and the use of top-level Dask functions such as dask.visualize() on such collections

* Understand the ability to use Dask transparently in all built-in Xarray operations

# Imports

For this tutorial, as we are working with Dask, there are a number of Dask packages that must be imported. Also, this is technically an Xarray tutorial, so Xarray and NumPy must also be imported. Finally, the Pythia datasets package is imported, allowing access to the Project Pythia example data library.

In [1]:
import dask
import dask.array as da
import numpy as np
import xarray as xr
from dask.diagnostics import ProgressBar
from dask.utils import format_bytes


## Blocked algorithms

As described above, the definition of “blocked algorithm” is an algorithm that replaces a large operation with many small operations. In the case of datasets, this means that a blocked algorithm separates a dataset into chunks, and performs an operation on each.

As an example of how blocked algorithms work, consider a dataset containing a billion numbers, and assume that the sum of the numbers is needed. Using a non-blocked algorithm, all of the numbers are added in one operation, which is extremely inefficient. However, by using a blocked algorithm, the dataset is broken into chunks. (For the purposes of this example, assume that 1,000 chunks are created, with 1,000,000 numbers each.) The sum of the numbers in each chunk is taken, most likely in parallel, and then each of those sums are summed to obtain the final result.

By using blocked algorithms, we achieve the result, in this case one sum of one billion numbers, through the results of many smaller operations, in this case one thousand sums of one million numbers each. (Also note that each of the one thousand sums must then be summed, making the total number of sums 1,001.) This allows for a much greater degree of parallelism, potentially speeding up the code execution dramatically.

<span style='color:red'>dask.array</span> contains these algorithms

The main object type used in Dask is <span style='color:red'>dask.array</span>, which implements a subset of the <span style='color:red'>ndarray</span> (NumPy array) interface. However, unlike <span style='color:red'>ndarray, dask.array</span> uses blocked algorithms, which break up the array into smaller arrays, as described above. This allows for the execution of computations on arrays larger than memory, by using parallelism to divide the computation among multiple cores. Dask manages and coordinates blocked algorithms for any given computation by using Dask graphs, which lay out in detail the steps Dask takes to solve a problem. In addition, <span style='color:red'>dask.array</span> objects, known as Dask Arrays, are <b>lazy</b>; in other words, any computation performed on them is delayed until a specific method is called.

Create a <span style='color:red'>dask.array</span> object

As stated earlier, Dask Arrays are loosely based on NumPy arrays. In the next set of examples, we illustrate the main differences between Dask Arrays and NumPy arrays. In order to illustrate the differences, we must have both a Dask Array object and a NumPy array object. Therefore, this first example creates a 3-D NumPy array of random data:

In [2]:
shape = (600, 200, 200)
arr = np.random.random(shape)
arr

array([[[4.52520859e-01, 4.49957547e-01, 7.94812678e-01, ...,
         7.55284395e-01, 7.62022328e-01, 3.06030979e-01],
        [8.70718126e-01, 1.92785437e-01, 7.75553670e-01, ...,
         3.07114551e-01, 1.51262248e-01, 4.66646979e-01],
        [5.35983708e-01, 6.00710250e-01, 9.39496044e-01, ...,
         4.41391442e-02, 7.73076721e-01, 6.48369566e-01],
        ...,
        [9.73318853e-01, 1.19584053e-01, 3.92478874e-01, ...,
         3.13999810e-01, 9.81250709e-01, 7.47971432e-01],
        [8.11719218e-01, 1.59908608e-01, 9.89153682e-01, ...,
         9.54613518e-01, 7.47960930e-01, 6.62810857e-01],
        [3.91558390e-01, 4.08092056e-01, 5.11095959e-01, ...,
         3.81202316e-01, 3.69302832e-01, 4.60682465e-01]],

       [[5.29416775e-01, 7.47199149e-01, 7.50203217e-01, ...,
         7.82487697e-01, 3.53304461e-01, 8.67990638e-01],
        [6.80861104e-01, 1.86192649e-01, 5.14169594e-01, ...,
         2.66557151e-01, 3.13921693e-01, 2.48753061e-01],
        [3.60866191e-01, 

In [3]:
format_bytes(arr.nbytes)

'183.11 MiB'

As shown above, this NumPy array contains about 183 MB of data.

As stated above, we must also create a Dask Array. This next example creates a Dask Array with the same dimension sizes as the existing NumPy array:

In [4]:
darr = da.random.random(shape, chunks=(300, 100, 200))

By specifying values to the <span style='color:red'>chunks</span> keyword argument, we can specify the array pieces that Dask’s blocked algorithms break the array into; in this case, we specify <span style='color:red'>(300, 100, 200)</span>.

<b>Specifying Chunks</b>

In this tutorial, we specify Dask Array chunks in a block shape. However, there are many additional ways to specify chunks; see this documentation (https://docs.dask.org/en/latest/array-chunks.html) for more details.

If you are viewing this page as a Jupyter Notebook, the next Jupyter cell will produce a rich information graphic giving in-depth details about the array and each individual chunk.

In [5]:
darr

dask.array<random_sample, shape=(600, 200, 200), dtype=float64, chunksize=(300, 100, 200), chunktype=numpy.ndarray>

The above graphic contains a symbolic representation of the array, including <span style='color:red'>shape, dtype</span>, and <span style='color:red'>chunksize</span>. (Your view may be different, depending on how you are accessing this page.) Notice that there is no data shown for this array; this is because Dask Arrays are lazy, as described above. Before we call a compute method for this array, we first illustrate the structure of a Dask graph. In this example, we show the Dask graph by calling <span style='color:red'>.visualize()</span> on the array:

In [6]:
darr.visualize()

As shown in the above Dask graph, our array has four chunks, each one created by a call to NumPy’s “random” method (<span style='color:red'>np.random.random</span>). These chunks are concatenated into a single array after the calculation is performed.

### Manipulate a <span style='color:red'>dask.array object as you would a numpy array

We can perform computations on the Dask Array created above in a similar fashion to NumPy arrays. These computations include arithmetic, slicing, and reductions, among others.

Although the code for performing these computations is similar between NumPy arrays and Dask Arrays, the process by which they are performed is quite different. For example, it is possible to call <span style='color:red'>sum() on both a NumPy array and a Dask Array; however, these two <span style='color:red'>sum() calls are definitely not the same, as shown below.

#### What’s the difference?
When <span style='color:red'>sum()</span> is called on a Dask Array, the computation is not performed; instead, an expression of the computation is built. The <span style='color:red'>sum()</span> computation, as well as any other computation methods called on the same Dask Array, are not performed until a specific method (known as a compute method) is called on the array. (This is known as lazy execution.) On the other hand, calling <span style='color:red'>sum()</span> on a NumPy array performs the calculation immediately; this is known as eager execution.

#### Why the difference?
As described earlier, a Dask Array is divided into chunks. Any computations run on the Dask Array run on each chunk individually. If the result of the computation is obtained before the computation runs through all of the chunks, Dask can stop the computation to save CPU time and memory resources.

This example illustrates calling <span style='color:red'>sum()</span> on a Dask Array; it also includes a demonstration of lazy execution, as well as another Dask graph display:

In [7]:
total = darr.sum()
total

dask.array<sum-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [8]:
total.visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

#### Compute the result
As described above, Dask Array objects make use of lazy execution. Therefore, operations performed on a Dask Array wait to execute until a compute method is called. As more operations are queued in this way, the Dask Array’s Dask graph increases in complexity, reflecting the steps Dask will take to perform all of the queued operations.

In this example, we call a compute method, simply called <span style='color:red'>.compute()</span>, to run on the Dask Array all of the stored computations:

In [9]:
%%time
total.compute()

CPU times: total: 266 ms
Wall time: 324 ms


12002304.454298109

## Exercise with <span style='color:red'>dask.arrays

In this section of the page, the examples are hands-on exercises pertaining to Dask Arrays. If these exercises are not interesting to you, this section can be used strictly as examples regardless of how the page is viewed. However, if you wish to participate in the exercises, make sure that you are viewing this page as a Jupyter Notebook.

For the first exercise, modify the chunk size or shape of the Dask Array created earlier. Call <span style='color:red'>.sum()</span> on the modified Dask Array, and visualize the Dask graph to view the changes.

In [11]:
da.random.random(shape, chunks=(50, 200, 400)).sum().visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

As is obvious from the above exercise, Dask quickly and easily determines a strategy for performing the operations, in this case a sum. This illustrates the appeal of Dask: automatic algorithm generation that scales from simple arithmetic problems to highly complex scientific equations with large datasets and multiple operations.

In this next set of examples, we demonstrate that increasing the complexity of the operations performed also increases the complexity of the Dask graph.

In this example, we use randomly selected functions, arguments and Python slices to create a complex set of operations. We then visualize the Dask graph to illustrate the increased complexity:

In [13]:
z = darr.dot(darr.T).mean(axis=0)[::2, :].std(axis=1)
z

dask.array<_sqrt, shape=(100, 600), dtype=float64, chunksize=(50, 300), chunktype=numpy.ndarray>

In [15]:
z.visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

### Testing a bigger calculation

While the earlier examples in this tutorial described well the basics of Dask, the size of the data in those examples, about 180 MB, is far too small for an actual use of Dask.

In this example, we create a much larger array, more indicative of data actually used in Dask:

In [16]:
darr = da.random.random((4000, 100, 4000), chunks=(1000, 100, 500)).astype('float32')
darr

dask.array<astype, shape=(4000, 100, 4000), dtype=float32, chunksize=(1000, 100, 500), chunktype=numpy.ndarray>

The dataset created in the previous example is much larger, approximately 6 GB. Depending on how many programs are running on your computer, this may be greater than the amount of free RAM on your computer. However, as Dask is larger-than-memory, the amount of free RAM does not impede Dask’s ability to work on this dataset.

In this example, we again perform randomly selected operations, but this time on the much larger dataset. We also visualize the Dask graph, and then run the compute method. However, as computing complex functions on large datasets is inherently time-consuming, we show a progress bar to track the progress of the computation.

In [17]:
z = (darr + darr.T)[::2, :].mean(axis=2)

In [19]:
z.visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

In [20]:
with ProgressBar():
    computed_ds = z.compute()

[########################################] | 100% Completed | 14.95 s


## Dask Arrays with Xarray

While directly interacting with Dask Arrays can be useful on occasion, more often than not Dask Arrays are interacted with through Xarray. Since Xarray wraps NumPy arrays, and Dask Arrays contain most of the functionality of NumPy arrays, Xarray can also wrap Dask Arrays, allowing anyone with knowledge of Xarray to easily start using the Dask interface.

### Reading data with <span style='color:red'>Dask</span> and <span style='color:red'>Xarray</span>

As demonstrated in previous examples, a Dask Array consists of many smaller arrays, called chunks:

In [21]:
darr

dask.array<astype, shape=(4000, 100, 4000), dtype=float32, chunksize=(1000, 100, 500), chunktype=numpy.ndarray>

As shown in the following example, to read data into Xarray as Dask Arrays, simply specify the <span style='color:red'>chunks</span> keyword argument when calling the <span style='color:red'>open_dataset()</span> function:

In [23]:
ds = xr.open_dataset('./data/CESM2_sst_data.nc', chunks={})
ds.tos

C:\Users\olaba\anaconda3\envs\pythia\lib\site-packages\xarray\conventions.py:523: SerializationWarning: variable 'tos' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


<xarray.DataArray 'tos' (time: 180, lat: 180, lon: 360)>
dask.array<open_dataset-d38cad3d5eeaa7c117c91faaa16fa946tos, shape=(180, 180, 360), dtype=float32, chunksize=(180, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2000-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
Attributes: (12/19)
    cell_measures:  area: areacello
    cell_methods:   area: mean where sea time: mean
    comment:        Model data on the 1x1 grid includes values in all cells f...
    description:    This may differ from "surface temperature" in regions of ...
    frequency:      mon
    id:             tos
    ...             ...
    time_label:     time-mean
    time_title:     Temporal mean
    title:          Sea Surface Temperature
    type:           real
    units:          degC
    variable_id:    tos

Correct usage of the <span style='color:red'>chunks</span> keyword argument specifies how many values in each dimension are contained in a single chunk. In this example, specifying the chunks keyword argument as <span style='color:red'>chunks={'time':90}</span> indicates to Xarray and Dask that 90 time slices are allocated to each chunk on the temporal axis.

Since this dataset contains 180 total time slices, the data variable <span style='color:red'>tos</span> (holding the sea surface temperature data) is now split into two chunks in the temporal dimension.

In [25]:
ds = xr.open_dataset(
    './data/CESM2_sst_data.nc',
    engine="netcdf4",
    chunks={"time": 90, "lat": 180, "lon": 360},
)
ds.tos

C:\Users\olaba\anaconda3\envs\pythia\lib\site-packages\xarray\conventions.py:523: SerializationWarning: variable 'tos' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


<xarray.DataArray 'tos' (time: 180, lat: 180, lon: 360)>
dask.array<open_dataset-b59745a04f92fa32b9d131050b743e05tos, shape=(180, 180, 360), dtype=float32, chunksize=(90, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2000-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
Attributes: (12/19)
    cell_measures:  area: areacello
    cell_methods:   area: mean where sea time: mean
    comment:        Model data on the 1x1 grid includes values in all cells f...
    description:    This may differ from "surface temperature" in regions of ...
    frequency:      mon
    id:             tos
    ...             ...
    time_label:     time-mean
    time_title:     Temporal mean
    title:          Sea Surface Temperature
    type:           real
    units:          degC
    variable_id:    tos

It is fairly straightforward to retrieve a list of the chunks and their sizes for each dimension; simply call the <span style='color:red'>.chunks</span> method on an Xarray <span style='color:red'>DataArray</span>. In this example, we show that the <span style='color:red'>tos DataArray</span> now contains two chunks on the <span style='color:red'>time</span> dimension, with each chunk containing 90 time slices.

In [26]:
ds.tos.chunks

((90, 90), (180,), (360,))

#### Xarray data structures are first-class dask collections
If an Xarray Dataset or DataArray object uses a Dask Array, rather than a NumPy array, it counts as a first-class Dask collection. This means that you can pass such an object to dask.visualize() and dask.compute(), in the same way as an individual Dask Array.

In this example, we call dask.visualize on our Xarray DataArray, displaying a Dask graph for the DataArray object:

In [27]:
dask.visualize(ds)

### Parallel and lazy computation using dask.array with Xarray
As described above, Xarray Datasets and DataArrays containing Dask Arrays are first-class Dask collections. Therefore, computations performed on such objects are deferred until a compute method is called. (This is the definition of lazy computation.)

In [28]:
z = ds.tos.mean(['lat', 'lon']).dot(ds.tos.T)
z

<xarray.DataArray 'tos' (lon: 360, lat: 180)>
dask.array<sum-aggregate, shape=(360, 180), dtype=float32, chunksize=(360, 180), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5

As shown in the above example, the result of the applied operations is an Xarray DataArray that contains a Dask Array, an identical object type to the object that the operations were performed on. This is true for any operations that can be applied to Xarray DataArrays, including subsetting operations; this next example illustrates this:

In [29]:
z.isel(lat=0)

<xarray.DataArray 'tos' (lon: 360)>
dask.array<getitem, shape=(360,), dtype=float32, chunksize=(360,), chunktype=numpy.ndarray>
Coordinates:
    lat      float64 -89.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5

Because the data subset created above is also a first-class Dask collection, we can view its Dask graph using the dask.visualize() function, as shown in this example:

In [31]:
dask.visualize(z)

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

Since this object is a first-class Dask collection, the computations performed on it have been deferred. To run these computations, we must call a compute method, in this case .compute(). This example also uses a progress bar to track the computation progress.

In [32]:
with ProgressBar():
    computed_ds = z.compute()

[########################################] | 100% Completed | 222.85 ms


# Summary
This tutorial covered the use of Xarray to access Dask Arrays, and the use of the chunks keyword argument to open datasets with Dask data instead of NumPy data. Another important concept introduced in this tutorial is the usage of Xarray Datasets and DataArrays as Dask collections, allowing Xarray objects to be manipulated in a similar manner to Dask Arrays. Finally, the concepts of larger-than-memory datasets, lazy computation, and parallel computation, and how they relate to Xarray and Dask, were covered.

## Dask Shortcomings
Although Dask Arrays and NumPy arrays are generally interchangeable, NumPy offers some functionality that is lacking in Dask Arrays. The usage of Dask Array comes with the following relevant issues:

1. Operations where the resulting shape depends on the array values can produce erratic behavior, or fail altogether, when used on a Dask Array. If the operation succeeds, the resulting Dask Array will have unknown chunk sizes, which can cause other sections of code to fail.

2. Operations that are by nature difficult to parallelize or less useful on very large datasets, such as sort, are not included in the Dask Array interface. Some of these operations have supported versions that are inherently more intuitive to parallelize, such as topk.

3. Development of new Dask functionality is only initiated when such functionality is required; therefore, some lesser-used NumPy functions, such as np.sometrue, are not yet implemented in Dask. However, many of these functions can be added as community contributions, or have already been added in this manner.